## Introduction 

In [1]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import numpy as np
import pandas as pd
import datetime as dt
import pylab
from matplotlib import pyplot as plt
%matplotlib inline
import scipy as sc
from scipy.stats import ttest_ind
from scipy.stats import boxcox
import re
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import power_transform
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings(action="ignore")


In [69]:
fifa = pd.read_csv('2019fifadata.csv')
fifa = pd.DataFrame(fifa)
print(fifa.columns)
print(fifa.info())

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

We'll look at how to predict a player's wage using the [FIFA dataset](https://www.kaggle.com/karangadiya/fifa19) . I'll remove columns that don't provide useful information (Unnamed:0, ID, Name, Photo, Flag, Club Logo, and Real Face). As we have 18,207 entries and multiple columns are missing 48 values, my instinct is that these rows are missing information in these columns. As such, I'll remove rows missing information in 48 cells. 

Columns LS through RB are missing values in over two thousand cells and while it's not obvious what each variable represents (there's no information on the Kaggle page and it's not immediately clear on the sofifa.com page), they could contain valuable information. Each value is a string of a number plus another number, so I'm going to strip the addition sign and the second number, and populate these cells with the first number. I will then fill the missing values with the mean.

Joined, Jersey Number, Loaned From, and Release Clause

After these updates, we're still missing values in Club, Contract Valid Until, and Position columns. While I could go through and find the information on each of these, I don't think that would be a great use of time for this project, so I'm going to drop rows for which there aren't values for Club/Contract valid until and hope this takes care of rows lacking a value in Position as well. Finally, Club is a categorical variable with over 650 categories, which will be a pain to create/utilize dummies for, so I will remove this column.

We've managed to clean up our data and keep over 98% of the rows. If we have trouble coming up with a model, we'll look at adding back some columns we've removed.

In [71]:
fifa = fifa.dropna(thresh=48, axis=0)

variables = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB']
fifa[variables].loc[:, variables[0]][0]

# for i, row in fifa.iterrows():
#     for variable in variables:
#         if(pd.isnull(row[variable])):
#             fifa.loc[i, variable] = fifa[variable].astype(float).mean()

# for i, row in fifa.iterrows():
#     for variable in variables:
#         try:
#           fifa.iloc[i, fifa.columns.get_loc(variable)] = fifa.iloc[i, fifa.columns.get_loc(variable)].apply(lambda x: x.split('+')[0])
#         except ValueError: 
#             pass        
#     if(pd.isnull(row[variable])):
#         fifa.loc[i, variable] = fifa[variable].astype(float).mean()

pd.options.display.max_columns = None
fifa.head()
# fifa.LS

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,1-Jul-04,NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,10-Jul-18,NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,3-Aug-17,NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,1-Jul-11,NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,30-Aug-15,NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [4]:
fifa[['LS', 'ST', 'RB']]

,LS,ST,RB
0,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
1,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
2,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
3,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
4,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
5,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
6,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
7,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
8,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>
9,<function <lambda> at 0x0BEDC300>,<function <lambda> at 0x0BEDC2B8>,<function <lambda> at 0x0BEDC348>


In [6]:
fifa = fifa[pd.notnull(fifa['Position'])]
fifa = fifa.drop(['Club', 'Contract Valid Until'], axis=1)
fifa.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18147 entries, 0 to 18206
Data columns (total 87 columns):
Unnamed: 0                  18147 non-null int64
ID                          18147 non-null int64
Name                        18147 non-null object
Age                         18147 non-null int64
Photo                       18147 non-null object
Nationality                 18147 non-null object
Flag                        18147 non-null object
Overall                     18147 non-null int64
Potential                   18147 non-null int64
Club Logo                   18147 non-null object
Value                       18147 non-null object
Wage                        18147 non-null object
Special                     18147 non-null int64
Preferred Foot              18147 non-null object
International Reputation    18147 non-null float64
Weak Foot                   18147 non-null float64
Skill Moves                 18147 non-null float64
Work Rate                   18147 non-null o

## Transform Variables

There are quite a few columns that are currently being reated as objects but should be integers or floats. I'll remove the €, K, and M from the Wage and Value columns and update the values as necessary. I will update the Height column into inches and remove the lbs from the Weight column. I'll perform one hot encoding on the remaining values (Nationality, Preferred Foot Work Rate, Body Type, and Position) to obtain dummies.

In [20]:
# fifa['Value'] = fifa['Value'].str.replace('€', '')
# fifa['Value'] = fifa['Value'].str.replace('M', '000000').astype(float)
# fifa['Value'] = fifa['Value'].str.replace('K', '000').astype(float)
# fifa['Wage'] = fifa['Wage'].str.replace('€', '')
# fifa['Wage'] = fifa['Wage'].str.replace('M', '000000').astype(float)
# fifa['Wage'] = fifa['Wage'].str.replace('K', '000').astype(float)

fifa['Value'] = fifa['Value'].str.replace('€', '')
fifa['Value'] = fifa['Value'].apply(lambda row: row.replace('K', '000') if 'K' in row else row.replace('M', '000000')).astype(float)

fifa['Wage'] = fifa['Wage'].str.replace('€', '')
fifa['Wage'] = fifa['Wage'].apply(lambda row: row.replace('K', '000') if 'K' in row else row.replace('M', '000000')).astype(float)

In [21]:
def parse_ht(ht):
    # format: 7' 0.0"
    feet = float(ht[0])
    inches = float(ht[2:])
    return 12*feet + inches


fifa['Height'] = fifa['Height'].apply(lambda x: parse_ht(x))


In [44]:
# fifa['Weight'] = fifa['Weight'].str.split('l')[0]
fifa['Weight'] = fifa['Weight'].apply(lambda x: float(str(x).replace('lbs', '')))

In [45]:
fifa.head()

,Age,Nationality,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,Argentina,94,94,110.5,565000.0,2202,Left,5.0,4.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,Portugal,94,94,77000000.0,405000.0,2228,Right,5.0,4.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,Brazil,92,93,118.5,290000.0,2143,Right,5.0,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,Spain,91,93,72000000.0,260000.0,1471,Right,4.0,3.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,Belgium,91,92,102000000.0,355000.0,2281,Right,4.0,5.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [22]:
fifa = pd.concat([fifa, pd.get_dummies(fifa['Nationality'])], axis=1)
fifa = pd.concat([fifa, pd.get_dummies(fifa["Preferred Foot"])], axis=1)
fifa = pd.concat([fifa, pd.get_dummies(fifa["Work Rate"])], axis=1)
fifa = pd.concat([fifa, pd.get_dummies(fifa["Body Type"])], axis=1)
fifa = pd.concat([fifa, pd.get_dummies(fifa["Position"])], axis=1)

# fifa.head()

In [23]:
corr_mat = np.abs((fifa.corr().loc[:, ['Wage']])).sort_values(by='Wage', ascending=False)
corr_mat[:50]

,Wage
Wage,1.000000
International Reputation,0.668635
Value,0.603267
Overall,0.571720
Reactions,0.495560
Potential,0.486604
Unnamed: 0,0.479437
Composure,0.419597
Special,0.347878
Vision,0.315395


In [25]:
# fifa = pd.get_dummies(fifa)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
fifa.corr()

In [ ]:
# Specify a list with features you want to test for normality. (You should ideally do this for all features in your dataset!)
features_to_test = ["norm_sqrt_Wage", "norm_winsorized_Overall", "norm_sqrt_Potential", "norm_sqrt_Value",
                    "norm_sqrt_International_Reputation", "norm_winsorized_Reactions", "norm_winsorized_Composure"]

# Now run each and one of the featues in above list. Locate the features in your data and calculate the statistics. 
for feat in features_to_test:
    jb_stats = stats.jarque_bera(fifa[feat])
    norm_stats = stats.normaltest(fifa[feat])
    
    print("Testing for normality on {}:".format(feat))
    print("Jarque-Bera test statistics for normalized Wage is {0} and p value is {1}".format(jb_stats[0], jb_stats[1]))
    print("Normality test statistics for normalized Wage is {0} and p value is {1}\n".format(norm_stats[0], norm_stats[1]))

In [ ]:
# Specify a list with features I want to test for normality. 
# Visuals for outliers and normality
for feat in features_to_test:
    winsorized_feature = winsorize(movies[feat], (0, 0.10))
    log_feature = np.log(movies[feat])
    sqrt_feature = np.sqrt(movies[feat])
    try:
        bc_feature,_ = stats.boxcox(movies[feat])
    except ValueError: 
        print(feat, 'Data isn\'t positive.')
    
    print('\n Outlier Visualization for', feat, 'feature. \n')
    plt.figure(figsize = (15,10))
    plt.subplot(3, 2, 1)
    sns.boxenplot(x=feat, data=movies)
    plt.title('Original')
    print('\n')
    plt.subplot(3, 2, 2)
    sns.distplot(movies[feat])
    plt.title('Original Distribution')
    print('\n')
    plt.subplot(3, 2, 3)
    sns.boxenplot(winsorized_feature, color='red')
    plt.title('Winsorized Feature')
    plt.subplot(3, 2, 4)
    sns.boxenplot(log_feature, color='green')
    plt.title('Log Feature')
    print('\n')
    plt.subplot(3, 2, 5)
    sns.boxenplot(sqrt_feature, color='orange')
    plt.title('Square Rooted Feature')
    plt.subplot(3, 2, 6)
    sns.boxenplot(bc_feature, color='pink')
    plt.title('Box-Cox Transformed Feature')
    plt.subplots_adjust(hspace=0.35, wspace=0.35)
    plt.show()

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfr = ensemble.RandomForestRegressor(n_estimators=200)
Y = fifa['Wage']
X = fifa[["Potential", "Value",  "Overall", "Reactions", "Composure", 'International Reputation']]
# X = pd.get_dummies(X)
# X = X.dropna(axis=1)

# rfr.fit(X, Y)
cross_val_score(rfr, X, Y, cv=10)


print(rfr.feature_importances_)